In [ ]:
import pandas as pd

data=pd.read_csv('combined_dataset5.csv')

In [ ]:
data.head()

In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

data = pd.read_csv('combined_dataset5.csv')

# 모델 및 토크나이저 로드
model_name = "gpt2"  # 또는 다른 모델명
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# 모델의 최대 위치 값을 늘릴 새로운 설정 정의
new_max_position_embeddings = 4096

# 기존 설정을 새로운 값으로 업데이트
config = model.config
config.max_position_embeddings = new_max_position_embeddings

# 모델을 새로운 설정으로 초기화
model = GPT2LMHeadModel(config)

def summarize_lyrics(lyrics_series):
    lyrics = lyrics_series['가사']

    # 입력 토큰화
    inputs = tokenizer(lyrics, return_tensors="pt", max_length=new_max_position_embeddings, truncation=True)

    # position_ids 값 조정
    inputs['position_ids'] = torch.arange(inputs['input_ids'].size(1)).unsqueeze(0)

    # 모델에 입력하고 출력 받기
    output = model.generate(
        **inputs,
        max_new_tokens=300,
        num_beams=5,
        length_penalty=0.6,
        no_repeat_ngram_size=2,
        top_k=50,
        temperature=0.85,
        do_sample=True
    )

    # 토큰을 텍스트로 디코딩
    summaries = tokenizer.batch_decode(output, skip_special_tokens=True)

    return summaries

# 상위 5개 노래만 요약 확인
sample_data = data.head()
sample_data['요약된_가사'] = sample_data.apply(summarize_lyrics, axis=1)
print(sample_data[['노래제목', '가수', '가사', '요약된_가사', '카테고리']])